# Model-level Bagging of Hyperbox-based Learners with Hyper-parameter Optimisation

This example shows how to use a Bagging classifier with a combination at the model level to generate a single model from many base learners, in which each base learner is trained by random search-based hyper-parameter tuning and k-fold cross-validation.

While the original model-level combination bagging classifier in the class ModelCombinationBagging uses the same base learners with the same hyperparameters, the cross-validation model-level combination bagging classifier in the class ModelCombinationCrossValBagging allows each base learner to use specific hyperparameters depending on the training data by performing random research to find the best combination of hyperparameters for each base learner.

In [1]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from hbbrain.numerical_data.ensemble_learner.model_comb_cross_val_bagging import ModelCombinationCrossValBagging
from hbbrain.numerical_data.incremental_learner.onln_gfmm import OnlineGFMM
from hbbrain.numerical_data.batch_learner.accel_agglo_gfmm import AccelAgglomerativeLearningGFMM

## Load dataset. This example will use the breast cancer dataset available in sklearn to demonstrate how to use this ensemble classifier. 

In [2]:
from sklearn.datasets import load_breast_cancer
from sklearn.preprocessing import MinMaxScaler

In [3]:
df = load_breast_cancer()
X = df.data
y = df.target

In [4]:
# Normailise data into the range of [0, 1] as hyperbox-based models only work in the unit cube
scaler = MinMaxScaler()
X = scaler.fit_transform(X)

In [5]:
# Split data into training, validation and testing sets
Xtr_val, X_test, ytr_val, y_test = train_test_split(X, y, train_size=0.8, random_state=0)
Xtr, X_val, ytr, y_val = train_test_split(X, y, train_size=0.75, random_state=0)

## This example will use the GFMM classifier with the original online learning algorithm as base learners. However, any type of hyperbox-based learning algorithms in this library can also be used to train base learners.

## 1. Using random subsampling to generate training sets for various base learners

### a. Training without pruning for base learners

In [6]:
# Initialise parameters
n_estimators = 20 # number of base learners
max_samples = 0.5 # sampling rate for samples
bootstrap = False # random subsampling without replacement
class_balanced = False # do not use the class-balanced sampling mode
n_jobs = 4 # number of processes is used to build base learners
n_iter = 20 # Number of parameter settings that are randomly sampled to choose the best combination of hyperparameters
k_fold = 5 # Number of folds to conduct Stratified K-Fold cross-validation for hyperparameter tunning

In [7]:
# Init a hyperbox-based model used to train base learners
# Using the GFMM classifier with the original online learning algorithm
base_estimator = OnlineGFMM()

In [8]:
# Init ranges for hyperparameters of base learners to perform a random search process for hyperparameter tunning
base_estimator_params = {'theta': np.arange(0.05, 1.01, 0.05), 'theta_min':[1], 'gamma':[0.5, 1, 2, 4, 8, 16]}

In [9]:
# Init a hyperbox-based model used to aggregate the resulting hyperboxes from all base learners
# Using the accelerated agglomerative learning algorithm for the GFMM model to do this task
model_level_estimator = AccelAgglomerativeLearningGFMM(theta=0.1, min_simil=0, simil_measure='long')

In [10]:
model_comb_cv_bagging_subsampling = ModelCombinationCrossValBagging(base_estimator=base_estimator, base_estimator_params=base_estimator_params, model_level_estimator=model_level_estimator, n_estimators=n_estimators, max_samples=max_samples, bootstrap=bootstrap, class_balanced=class_balanced, n_iter=n_iter, k_fold=k_fold, n_jobs=n_jobs, random_state=0)
model_comb_cv_bagging_subsampling.fit(Xtr, ytr)

ModelCombinationCrossValBagging(base_estimator=OnlineGFMM(C=array([], dtype=float64),
                                                          V=array([], dtype=float64),
                                                          W=array([], dtype=float64)),
                                base_estimator_params={'gamma': [0.5, 1, 2, 4,
                                                                 8, 16],
                                                       'theta': array([0.05, 0.1 , 0.15, 0.2 , 0.25, 0.3 , 0.35, 0.4 , 0.45, 0.5 , 0.55,
       0.6 , 0.65, 0.7 , 0.75, 0.8 , 0.85, 0.9 , 0.95, 1.  ]),
                                                       'theta_min': [1]},
                                model_level_estimator=AccelAgglomerativeLearningGFMM(min_simil=0,
                                                                                     simil_measure='long',
                                                                                     theta=0.1),
             

In [11]:
print("Training time: %.3f (s)"%(model_comb_cv_bagging_subsampling.elapsed_training_time))

Training time: 44.155 (s)


In [12]:
print('Total number of hyperboxes in all base learners = %d'%model_comb_cv_bagging_subsampling.get_n_hyperboxes())

Total number of hyperboxes in all base learners = 1168


In [13]:
print('Number of hyperboxes in the combined model = %d'%model_comb_cv_bagging_subsampling.get_n_hyperboxes_comb_model())

Number of hyperboxes in the combined model = 779


### Prediction

#### Using majority voting from predicted results of all base learners

In [14]:
y_pred_voting = model_comb_cv_bagging_subsampling.predict_voting(X_test)

In [15]:
acc_voting = accuracy_score(y_test, y_pred_voting)
print(f'Testing accuracy using voting of decisions from base learners = {acc_voting * 100 : .2f}%')

Testing accuracy using voting of decisions from base learners =  94.74%


#### Using the final combined single model to make prediction

In [16]:
y_pred = model_comb_cv_bagging_subsampling.predict(X_test)
acc = accuracy_score(y_test, y_pred)
print(f'Testing accuracy of the combined model = {acc * 100: .2f}%')

Testing accuracy of the combined model =  88.60%


### Apply pruning for the final combined model

In [17]:
acc_threshold=0.5 # minimum accuracy score of the unpruned hyperboxes
keep_empty_boxes=False # False means hyperboxes that do not join the prediction process within the pruning procedure are also eliminated
model_comb_cv_bagging_subsampling.simple_pruning(X_val, y_val, acc_threshold, keep_empty_boxes)

ModelCombinationCrossValBagging(base_estimator=OnlineGFMM(C=array([], dtype=float64),
                                                          V=array([], dtype=float64),
                                                          W=array([], dtype=float64)),
                                base_estimator_params={'gamma': [0.5, 1, 2, 4,
                                                                 8, 16],
                                                       'theta': array([0.05, 0.1 , 0.15, 0.2 , 0.25, 0.3 , 0.35, 0.4 , 0.45, 0.5 , 0.55,
       0.6 , 0.65, 0.7 , 0.75, 0.8 , 0.85, 0.9 , 0.95, 1.  ]),
                                                       'theta_min': [1]},
                                model_level_estimator=AccelAgglomerativeLearningGFMM(min_simil=0,
                                                                                     simil_measure='long',
                                                                                     theta=0.1),
             

In [18]:
print('Number of hyperboxes of the combined single model after pruning = %d'%model_comb_cv_bagging_subsampling.get_n_hyperboxes_comb_model())

Number of hyperboxes of the combined single model after pruning = 36


### Prediction after doing a pruning procedure for the combined single model

In [19]:
y_pred_2 = model_comb_cv_bagging_subsampling.predict(X_test)
acc_pruned = accuracy_score(y_test, y_pred_2)
print(f'Testing accuracy after pruning the final model = {acc_pruned * 100: .2f}%')

Testing accuracy after pruning the final model =  88.60%


### b. Training with pruning for base learners

In [20]:
model_comb_cv_bagging_subsampling_base_learner_pruning = ModelCombinationCrossValBagging(base_estimator=base_estimator, base_estimator_params=base_estimator_params, model_level_estimator=model_level_estimator, n_estimators=n_estimators, max_samples=max_samples, bootstrap=bootstrap, class_balanced=class_balanced, n_iter=n_iter, k_fold=k_fold, n_jobs=n_jobs, random_state=0)
model_comb_cv_bagging_subsampling_base_learner_pruning.fit(Xtr, ytr, is_pruning_base_learners=True, X_val=X_val, y_val=y_val, acc_threshold=acc_threshold, keep_empty_boxes=keep_empty_boxes)

ModelCombinationCrossValBagging(base_estimator=OnlineGFMM(C=array([], dtype=float64),
                                                          V=array([], dtype=float64),
                                                          W=array([], dtype=float64)),
                                base_estimator_params={'gamma': [0.5, 1, 2, 4,
                                                                 8, 16],
                                                       'theta': array([0.05, 0.1 , 0.15, 0.2 , 0.25, 0.3 , 0.35, 0.4 , 0.45, 0.5 , 0.55,
       0.6 , 0.65, 0.7 , 0.75, 0.8 , 0.85, 0.9 , 0.95, 1.  ]),
                                                       'theta_min': [1]},
                                model_level_estimator=AccelAgglomerativeLearningGFMM(min_simil=0,
                                                                                     simil_measure='long',
                                                                                     theta=0.1),
             

In [21]:
print("Training time: %.3f (s)"%(model_comb_cv_bagging_subsampling_base_learner_pruning.elapsed_training_time))

Training time: 44.437 (s)


In [22]:
print('Total number of hyperboxes in all base learners = %d'%model_comb_cv_bagging_subsampling_base_learner_pruning.get_n_hyperboxes())

Total number of hyperboxes in all base learners = 756


In [23]:
print('Number of hyperboxes in the combined model = %d'%model_comb_cv_bagging_subsampling_base_learner_pruning.get_n_hyperboxes_comb_model())

Number of hyperboxes in the combined model = 613


### Prediction

#### Using majority voting from predicted results of all base learners

In [24]:
y_pred_voting = model_comb_cv_bagging_subsampling_base_learner_pruning.predict_voting(X_test)

In [25]:
acc_voting = accuracy_score(y_test, y_pred_voting)
print(f'Testing accuracy using voting of decisions from base learners = {acc_voting * 100 : .2f}%')

Testing accuracy using voting of decisions from base learners =  96.49%


#### Using the final combined single model to make prediction

In [26]:
y_pred = model_comb_cv_bagging_subsampling_base_learner_pruning.predict(X_test)
acc = accuracy_score(y_test, y_pred)
print(f'Testing accuracy of the combined model = {acc * 100: .2f}%')

Testing accuracy of the combined model =  88.60%


### Apply pruning for the final combined model

In [27]:
acc_threshold=0.5 # minimum accuracy score of the unpruned hyperboxes
keep_empty_boxes=False # False means hyperboxes that do not join the prediction process within the pruning procedure are also eliminated
model_comb_cv_bagging_subsampling_base_learner_pruning.simple_pruning(X_val, y_val, acc_threshold, keep_empty_boxes)

ModelCombinationCrossValBagging(base_estimator=OnlineGFMM(C=array([], dtype=float64),
                                                          V=array([], dtype=float64),
                                                          W=array([], dtype=float64)),
                                base_estimator_params={'gamma': [0.5, 1, 2, 4,
                                                                 8, 16],
                                                       'theta': array([0.05, 0.1 , 0.15, 0.2 , 0.25, 0.3 , 0.35, 0.4 , 0.45, 0.5 , 0.55,
       0.6 , 0.65, 0.7 , 0.75, 0.8 , 0.85, 0.9 , 0.95, 1.  ]),
                                                       'theta_min': [1]},
                                model_level_estimator=AccelAgglomerativeLearningGFMM(min_simil=0,
                                                                                     simil_measure='long',
                                                                                     theta=0.1),
             

In [28]:
print('Number of hyperboxes of the combined single model after pruning = %d'%model_comb_cv_bagging_subsampling_base_learner_pruning.get_n_hyperboxes_comb_model())

Number of hyperboxes of the combined single model after pruning = 36


### Prediction after doing a pruning procedure for the combined single model

In [29]:
y_pred_2 = model_comb_cv_bagging_subsampling_base_learner_pruning.predict(X_test)
acc_pruned = accuracy_score(y_test, y_pred_2)
print(f'Testing accuracy after pruning the final model = {acc_pruned * 100: .2f}%')

Testing accuracy after pruning the final model =  88.60%


## 2. Using random undersampling to generate class-balanced training sets for various base learners

### a. Training without pruning for base learners

In [30]:
# Initialise parameters
n_estimators = 20 # number of base learners
max_samples = 0.5 # sampling rate for samples
bootstrap = False # random subsampling without replacement
class_balanced = True # use the class-balanced sampling mode
n_jobs = 4 # number of processes is used to build base learners
n_iter = 20 # Number of parameter settings that are randomly sampled to choose the best combination of hyperparameters
k_fold = 5 # Number of folds to conduct Stratified K-Fold cross-validation for hyperparameter tunning

In [31]:
# Init a hyperbox-based model used to train base learners
# Using the GFMM classifier with the original online learning algorithm
base_estimator = OnlineGFMM()

In [32]:
# Init ranges for hyperparameters of base learners to perform a random search process for hyperparameter tunning
base_estimator_params = {'theta': np.arange(0.05, 1.01, 0.05), 'theta_min':[1], 'gamma':[0.5, 1, 2, 4, 8, 16]}

In [33]:
# Init a hyperbox-based model used to aggregate the resulting hyperboxes from all base learners
# Using the accelerated agglomerative learning algorithm for the GFMM model to do this task
model_level_estimator = AccelAgglomerativeLearningGFMM(theta=0.1, min_simil=0, simil_measure='long')

In [34]:
model_comb_cv_bagging_class_balanced = ModelCombinationCrossValBagging(base_estimator=base_estimator, base_estimator_params=base_estimator_params, model_level_estimator=model_level_estimator, n_estimators=n_estimators, max_samples=max_samples, bootstrap=bootstrap, class_balanced=class_balanced, n_iter=n_iter, k_fold=k_fold, n_jobs=n_jobs, random_state=0)
model_comb_cv_bagging_class_balanced.fit(Xtr, ytr)

ModelCombinationCrossValBagging(base_estimator=OnlineGFMM(C=array([], dtype=float64),
                                                          V=array([], dtype=float64),
                                                          W=array([], dtype=float64)),
                                base_estimator_params={'gamma': [0.5, 1, 2, 4,
                                                                 8, 16],
                                                       'theta': array([0.05, 0.1 , 0.15, 0.2 , 0.25, 0.3 , 0.35, 0.4 , 0.45, 0.5 , 0.55,
       0.6 , 0.65, 0.7 , 0.75, 0.8 , 0.85, 0.9 , 0.95, 1.  ]),
                                                       'theta_min': [1]},
                                class_balanced=True,
                                model_level_estimator=AccelAgglomerativeLearningGFMM(min_simil=0,
                                                                                     simil_measure='long',
                                                         

In [35]:
print("Training time: %.3f (s)"%(model_comb_cv_bagging_class_balanced.elapsed_training_time))

Training time: 36.885 (s)


In [36]:
print('Total number of hyperboxes in all base learners = %d'%model_comb_cv_bagging_class_balanced.get_n_hyperboxes())

Total number of hyperboxes in all base learners = 1407


In [37]:
print('Number of hyperboxes in the combined model = %d'%model_comb_cv_bagging_class_balanced.get_n_hyperboxes_comb_model())

Number of hyperboxes in the combined model = 812


### Prediction

#### Using majority voting from predicted results of all base learners

In [38]:
y_pred_voting = model_comb_cv_bagging_class_balanced.predict_voting(X_test)

In [39]:
acc_voting = accuracy_score(y_test, y_pred_voting)
print(f'Testing accuracy using voting of decisions from base learners = {acc_voting * 100 : .2f}%')

Testing accuracy using voting of decisions from base learners =  94.74%


#### Using the final combined single model to make prediction

In [40]:
y_pred = model_comb_cv_bagging_class_balanced.predict(X_test)
acc = accuracy_score(y_test, y_pred)
print(f'Testing accuracy of the combined model = {acc * 100: .2f}%')

Testing accuracy of the combined model =  89.47%


### Apply pruning for the final combined model

In [41]:
acc_threshold=0.5 # minimum accuracy score of the unpruned hyperboxes
keep_empty_boxes=False # False means hyperboxes that do not join the prediction process within the pruning procedure are also eliminated
model_comb_cv_bagging_class_balanced.simple_pruning(X_val, y_val, acc_threshold, keep_empty_boxes)

ModelCombinationCrossValBagging(base_estimator=OnlineGFMM(C=array([], dtype=float64),
                                                          V=array([], dtype=float64),
                                                          W=array([], dtype=float64)),
                                base_estimator_params={'gamma': [0.5, 1, 2, 4,
                                                                 8, 16],
                                                       'theta': array([0.05, 0.1 , 0.15, 0.2 , 0.25, 0.3 , 0.35, 0.4 , 0.45, 0.5 , 0.55,
       0.6 , 0.65, 0.7 , 0.75, 0.8 , 0.85, 0.9 , 0.95, 1.  ]),
                                                       'theta_min': [1]},
                                class_balanced=True,
                                model_level_estimator=AccelAgglomerativeLearningGFMM(min_simil=0,
                                                                                     simil_measure='long',
                                                         

In [42]:
print('Number of hyperboxes of the combined single model after pruning = %d'%model_comb_cv_bagging_class_balanced.get_n_hyperboxes_comb_model())

Number of hyperboxes of the combined single model after pruning = 42


### Prediction after doing a pruning procedure for the combined single model

In [43]:
y_pred_2 = model_comb_cv_bagging_class_balanced.predict(X_test)
acc_pruned = accuracy_score(y_test, y_pred_2)
print(f'Testing accuracy after pruning the final model = {acc_pruned * 100: .2f}%')

Testing accuracy after pruning the final model =  90.35%


### b. Training with pruning for base learners

In [45]:
model_comb_cv_bagging_class_balanced_base_learner_pruning = ModelCombinationCrossValBagging(base_estimator=base_estimator, base_estimator_params=base_estimator_params, model_level_estimator=model_level_estimator, n_estimators=n_estimators, max_samples=max_samples, bootstrap=bootstrap, class_balanced=class_balanced, n_iter=n_iter, k_fold=k_fold, n_jobs=n_jobs, random_state=0)
model_comb_cv_bagging_class_balanced_base_learner_pruning.fit(Xtr, ytr, is_pruning_base_learners=True, X_val=X_val, y_val=y_val, acc_threshold=acc_threshold, keep_empty_boxes=keep_empty_boxes)

ModelCombinationCrossValBagging(base_estimator=OnlineGFMM(C=array([], dtype=float64),
                                                          V=array([], dtype=float64),
                                                          W=array([], dtype=float64)),
                                base_estimator_params={'gamma': [0.5, 1, 2, 4,
                                                                 8, 16],
                                                       'theta': array([0.05, 0.1 , 0.15, 0.2 , 0.25, 0.3 , 0.35, 0.4 , 0.45, 0.5 , 0.55,
       0.6 , 0.65, 0.7 , 0.75, 0.8 , 0.85, 0.9 , 0.95, 1.  ]),
                                                       'theta_min': [1]},
                                class_balanced=True,
                                model_level_estimator=AccelAgglomerativeLearningGFMM(min_simil=0,
                                                                                     simil_measure='long',
                                                         

In [46]:
print("Training time: %.3f (s)"%(model_comb_cv_bagging_class_balanced_base_learner_pruning.elapsed_training_time))

Training time: 31.609 (s)


In [47]:
print('Total number of hyperboxes in all base learners = %d'%model_comb_cv_bagging_class_balanced_base_learner_pruning.get_n_hyperboxes())

Total number of hyperboxes in all base learners = 719


In [48]:
print('Number of hyperboxes in the combined model = %d'%model_comb_cv_bagging_class_balanced_base_learner_pruning.get_n_hyperboxes_comb_model())

Number of hyperboxes in the combined model = 538


### Prediction

#### Using majority voting from predicted results of all base learners

In [50]:
y_pred_voting = model_comb_cv_bagging_class_balanced_base_learner_pruning.predict_voting(X_test)

In [51]:
acc_voting = accuracy_score(y_test, y_pred_voting)
print(f'Testing accuracy using voting of decisions from base learners = {acc_voting * 100 : .2f}%')

Testing accuracy using voting of decisions from base learners =  95.61%


#### Using the final combined single model to make prediction

In [52]:
y_pred = model_comb_cv_bagging_class_balanced_base_learner_pruning.predict(X_test)
acc = accuracy_score(y_test, y_pred)
print(f'Testing accuracy of the combined model = {acc * 100: .2f}%')

Testing accuracy of the combined model =  89.47%


### Apply pruning for the final combined model

In [53]:
acc_threshold=0.5 # minimum accuracy score of the unpruned hyperboxes
keep_empty_boxes=False # False means hyperboxes that do not join the prediction process within the pruning procedure are also eliminated
model_comb_cv_bagging_class_balanced_base_learner_pruning.simple_pruning(X_val, y_val, acc_threshold, keep_empty_boxes)

ModelCombinationCrossValBagging(base_estimator=OnlineGFMM(C=array([], dtype=float64),
                                                          V=array([], dtype=float64),
                                                          W=array([], dtype=float64)),
                                base_estimator_params={'gamma': [0.5, 1, 2, 4,
                                                                 8, 16],
                                                       'theta': array([0.05, 0.1 , 0.15, 0.2 , 0.25, 0.3 , 0.35, 0.4 , 0.45, 0.5 , 0.55,
       0.6 , 0.65, 0.7 , 0.75, 0.8 , 0.85, 0.9 , 0.95, 1.  ]),
                                                       'theta_min': [1]},
                                class_balanced=True,
                                model_level_estimator=AccelAgglomerativeLearningGFMM(min_simil=0,
                                                                                     simil_measure='long',
                                                         

In [54]:
print('Number of hyperboxes of the combined single model after pruning = %d'%model_comb_cv_bagging_class_balanced_base_learner_pruning.get_n_hyperboxes_comb_model())

Number of hyperboxes of the combined single model after pruning = 42


### Prediction after doing a pruning procedure for the combined single model

In [55]:
y_pred_2 = model_comb_cv_bagging_class_balanced_base_learner_pruning.predict(X_test)
acc_pruned = accuracy_score(y_test, y_pred_2)
print(f'Testing accuracy after pruning the final model = {acc_pruned * 100: .2f}%')

Testing accuracy after pruning the final model =  90.35%
